In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("data/TotBytes_Dur_discretized.csv")

In [13]:
from nltk import ngrams

In [51]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [55]:
import numpy as np

In [212]:
ohe_dfs = []
for (src, dst), indexes in df[["SrcAddr", "DstAddr", "TotBytes_Dur"]].groupby(["SrcAddr", "DstAddr"]).groups.items():
    group_df = df.loc[indexes]
    ng = set(ngrams(group_df["TotBytes_Dur"], 3))
    ng = list(map(str, ng))
    ohe_df = pd.get_dummies(pd.Series(ng))
    # ohe_df = ohe_df.assign(SrcAddr=src, DstAddr=dst)
    ohe_dfs.append(ohe_df)

<ipython-input-212-9d708b59720f>:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ohe_df = pd.get_dummies(pd.Series(ng))


In [217]:
profile_df = pd.concat(ohe_dfs).fillna(0)

In [218]:
profile_df

,"(3.0, 3.0, 3.0)","(8.0, 3.0, 3.0)","(3.0, 3.0, 8.0)","(3.0, 8.0, 3.0)","(8.0, 3.0, 8.0)","(8.0, 8.0, 3.0)","(8.0, 8.0, 8.0)","(13.0, 8.0, 3.0)","(3.0, 13.0, 8.0)","(8.0, 3.0, 13.0)",...,"(8.0, 16.0, 12.0)","(4.0, 7.0, 3.0)","(9.0, 4.0, 7.0)","(7.0, 9.0, 9.0)","(9.0, 7.0, 9.0)","(12.0, 12.0, 14.0)","(12.0, 14.0, 14.0)","(14.0, 14.0, 12.0)","(2.0, 4.0, 2.0)","(4.0, 2.0, 4.0)"
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [242]:
profile = profile_df.to_numpy()

### Transpose profile to have IPs as the columns

In [243]:
profile = profile.T

In [232]:
from itertools import permutations

In [249]:
# create permutations
perm = list(permutations([i for i in range(367)], 2))

In [253]:
from tqdm import tqdm

## Jaccard Similarity of columns

In [254]:
jacc = []
for p in tqdm(perm):
    p1 = profile[:, p[0]]
    p2 = profile[:, p[1]]
    intersection = 0
    union = 0
    for i in range(profile.shape[0]):
        if p1[i] == 1:
            union += 1
            if p2[i] == 1:
                intersection += 1
        elif p2[i] == 1:
            union += 1
            if p1[i] == 1:
                intersection += 1
    jacc.append(intersection/union)

100%|██████████| 134322/134322 [01:07<00:00, 1984.69it/s]


In [185]:
from collections import defaultdict

In [201]:
from math import ceil, log2
from primesieve import nth_prime
from random import randint

class UniversalHashing:
    """
    N = #bins
    p = prime number st: p >= N
    """
    def __init__(self, N, p = None):
        self.N = N
        if p is None:
            p = nth_prime(1, 1 << max(32, ceil(log2(N))))
        assert p >= N, 'Prime number p should be at least N!'
        self.p = p

    def draw(self):
        a = randint(1, self.p - 1)
        b = randint(0, self.p - 1)
        return lambda x: ((a * x + b) % self.p) % self.N

In [202]:
# bins
N = 50

In [222]:
num_hash_fns = 100
H = UniversalHashing(50)
hash_fns = []
for i in range(num_hash_fns):
    hash_fns.append(H.draw())
h = defaultdict(dict)
M = np.full([num_hash_fns, profile.shape[1]], np.inf)
for r in range(profile.shape[0]):
    # compute hash values
    for i, h_i in enumerate(hash_fns):
        # compute h_i(profile[r])
        h[i][r] = h_i(r)
    for c in range(profile.shape[1]):
        if profile[r, c] == 1:
            for i in range(num_hash_fns):
                if h[i][r] < M[i, c]:
                    M[i, c] = h[i][r]

## Signature Similarity

In [256]:
sig_sim = []
for p in tqdm(perm):
    p1 = M[:, p[0]]
    p2 = M[:, p[1]]
    sim = 0
    tot = M.shape[0]
    for i in range(M.shape[0]):
        if p1[i] == p2[i]:
            sim += 1
    sig_sim.append(sim/tot)

100%|██████████| 134322/134322 [00:08<00:00, 14990.84it/s]


In [258]:
import math

### Compute Error of estimation

In [260]:
errors = [math.sqrt((s - j)**2) for s, j in zip(sig_sim, jacc)]

In [265]:
sum(errors)/len(errors)

0.01604711067434256